https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

In [42]:
import tensorflow as tf

In [43]:
import keras

In [44]:
#import packages
import os
import pandas as pd
import numpy as np

df = pd.read_csv('/Users/mbagav200/Desktop/sample.csv') #Read excel file

In [45]:
df['target'].value_counts()

0    573518
1     21694
Name: target, dtype: int64

In [46]:
df = df.set_index('id')

In [47]:
# Split data into train and valid

from sklearn.cross_validation import train_test_split

train, test = train_test_split(df, test_size = 0.25)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [48]:
# Build prediction models
features_train = np.array(train[train.columns.difference(['target'])])
label_train = np.array(train['target'])
features_test = np.array(test[test.columns.difference(['target'])])
label_test = np.array(test['target'])

In [60]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [68]:
model = Sequential()
model.add(Dense(16,input_dim=16,activation='relu'))
model.add(Dense(8,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [69]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [70]:
model.fit(features_train,label_train,epochs=10,batch_size=500)

Epoch 1/10
446409/446409 [==============================] - 4s - loss: 0.1643 - acc: 0.9635     
Epoch 2/10
446409/446409 [==============================] - 3s - loss: 0.1561 - acc: 0.9635     
Epoch 3/10
446409/446409 [==============================] - 3s - loss: 0.1550 - acc: 0.9635     
Epoch 4/10
446409/446409 [==============================] - 3s - loss: 0.1546 - acc: 0.9635     
Epoch 5/10
446409/446409 [==============================] - 3s - loss: 0.1543 - acc: 0.9635     
Epoch 6/10
446409/446409 [==============================] - 3s - loss: 0.1541 - acc: 0.9635     
Epoch 7/10
446409/446409 [==============================] - 3s - loss: 0.1540 - acc: 0.9635     
Epoch 8/10
446409/446409 [==============================] - 3s - loss: 0.1539 - acc: 0.9635     
Epoch 9/10
446409/446409 [==============================] - 3s - loss: 0.1539 - acc: 0.9635     
Epoch 10/10
446409/446409 [==============================] - 3s - loss: 0.1537 - acc: 0.9635     


In [53]:
scores = model.evaluate(features_train, label_train)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

446048/446409 [============================>.] - ETA: 0s
acc: 96.35%


In [54]:
from collections import OrderedDict
import pandas as pd
import sys
%matplotlib inline
    
def plot_pandas_style(styler):
    from IPython.core.display import HTML
    html = '\n'.join([line.lstrip() for line in styler.render().split('\n')])
    return HTML(html)

def highlight_max(s,color='yellow'):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: {}'.format(color) if v else '' for v in is_max]

def decile_labels(agg1,label,color='skyblue'):
    agg_dummy = pd.DataFrame(OrderedDict((('TOTAL',0),('TARGET',0),('NONTARGET',0),('PCT_TAR',0),('CUM_TAR',0),('CUM_NONTAR',0),('DIST_TAR',0),('DIST_NONTAR',0),('SPREAD',0))),index=[0])
    agg1 = agg1.append(agg_dummy).sort_index()
    agg1.index.name = label
    agg1 = agg1.style.apply(highlight_max, color = 'yellow', subset=['SPREAD'])
    agg1.bar(subset=['TARGET'], color='{}'.format(color))
    agg1.bar(subset=['TOTAL'], color='{}'.format(color))
    agg1.bar(subset=['PCT_TAR'], color='{}'.format(color))
    return(agg1)

def deciling(data,decile_by,target,nontarget):
    inputs = list(decile_by)
    inputs.extend((target,nontarget))
    decile = data[inputs]
    grouped = decile.groupby(decile_by)
    agg1 = pd.DataFrame({},index=[])
    agg1['TOTAL'] = grouped.sum()[nontarget] + grouped.sum()[target]
    agg1['TARGET'] = grouped.sum()[target]
    agg1['NONTARGET'] = grouped.sum()[nontarget]
    agg1['PCT_TAR'] = grouped.mean()[target]*100
    agg1['CUM_TAR'] = grouped.sum()[target].cumsum()
    agg1['CUM_NONTAR'] = grouped.sum()[nontarget].cumsum()
    agg1['DIST_TAR'] = agg1['CUM_TAR']/agg1['TARGET'].sum()*100
    agg1['DIST_NONTAR'] = agg1['CUM_NONTAR']/agg1['NONTARGET'].sum()*100
    agg1['SPREAD'] = (agg1['DIST_TAR'] - agg1['DIST_NONTAR'])
    agg1 = decile_labels(agg1,'NATIONAL',color='skyblue')
    return(plot_pandas_style(agg1))

In [55]:
def scoring(features,clf,target):
    score = pd.DataFrame(clf.predict_proba(features), columns = ['SCORE'])
    score['DECILE'] = pd.qcut(score['SCORE'].rank(method = 'first'),10,labels=range(10,0,-1))
    score['DECILE'] = score['DECILE'].astype(float)
    score['TARGET'] = target
    score['NONTARGET'] = 1 - target
    return(score)

In [71]:
scores_train = scoring(features_train,model,label_train)
scores_test = scoring(features_test,model,label_test)

148224/148803 [============================>.] - ETA: 0s

In [72]:
deciling(scores_train,['DECILE'],'TARGET','NONTARGET')

In [73]:
deciling(scores_test,['DECILE'],'TARGET','NONTARGET')